In [1]:
from collections import defaultdict
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn import linear_model
from sets import Set


comments = []
usernames = []
#stop_words = set(stopwords.words('english'))
# Alternatively, just remove the basic stop words...
stop_words = {"a","an","and","are","as","at","be","by","for","from","has","he","in","is","it"
             ,"its","of","on","that","the","to","was","where","will","with"}
should_stem = 0
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

def stem(w):
    if(should_stem == 0):
        return w
    else:
        return stemmer.stem(w)
    
#creating a small test file for convenience
def read(f):
    for l in open(f):
        yield eval(l)
        
x = open("opt_dataset/opt.json")
i = 0
training = open("opt_dataset/small_opt.json",'w')

for l in x:
    if (i>=0) and (i<30000):
        training.write(l)
        if(i==0):
            print l
    i += 1
training.close()

# forming the initial vocabulary on the training set of 600 samples
# Training set: 
# Test set: 
# Not using stemming currently, need to modify later


for i in read('opt_dataset/small_opt.json'):
    comments.append(i['comment'])
    usernames.append(i['userName'])

def bigrams(words):
    if(should_stem == 0):
        return nltk.bigrams(words)
    else:
        for i in range(len(words)):
            words[i] = stem(words[i])
        return nltk.bigrams(words)

        
# extracing all the known labels
labels = []
for l in open('opt_dataset/labels_1-900.txt'):
    if(eval(l) == 1):
        labels.append(1)
    else:
        labels.append(0)

{'userName': 'Roxanne Liu', 'recievedDate': '11/08/2015', 'docID': 'ICEB-2015-0002-12058', 'comment': 'It is very enssential for America to keep the international students. They are epquiped with skills and knowledge. The development of America relies on talents. Competition should not be the reason to ask them to leave.', 'postedDate': '11/11/2015'}



/Library/Python/2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: the sets module is deprecated


In [2]:
# Multinomial naive bayes!
def fit_params(num_examples, data_labels,update_vocab):
    train_no = num_examples
    # extracting the vocab
    if(update_vocab == 1):
        vocab_no = train_no
    else:
        vocab_no = 700
        
    vocab = Set()
    for i in range(vocab_no):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        for w in r.split():
            if(w not in stop_words):
                w = stem(w)
                vocab.add(w)

    pos_wordCount_mult = defaultdict(int)
    neg_wordCount_mult = defaultdict(int)
    
    for w in vocab:
        pos_wordCount_mult[w] = 0
        neg_wordCount_mult[w] = 0
    numberpos_mult = 0.0
    numberneg_mult = 0.0
    posexamples = 0.0
    negexamples = 0.0
    # calculating the probabilites
    for i in range(train_no):
        if(data_labels[i] == 1):
            posexamples += 1
        else:
            negexamples += 1
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for w in bag:
            w = stem(w)
            if(w in vocab):
                if(data_labels[i] == 1):
                    pos_wordCount_mult[w] += 1
                    numberpos_mult += 1
                else:
                    neg_wordCount_mult[w] += 1
                    numberneg_mult += 1
                    
    posprob = posexamples/(posexamples + negexamples)
    negprob = negexamples/(posexamples + negexamples)
    
    pos_wordprob_mult = defaultdict(float)
    neg_wordprob_mult = defaultdict(float)

    for w in vocab:
        pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
        neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)
    
    return vocab, pos_wordprob_mult,neg_wordprob_mult,posprob, negprob

import math
def predpos_mult(words,vocab,prob, frac):
    sum = 0.0
    for w in words:
        w = stem(w)
        if(w in vocab):
            sum += math.log(prob[w])
    return (math.log(frac) + sum)
            
def predneg_mult(words, vocab, prob, frac):
    sum = 0.0
    for w in words:
        w = stem(w)
        if(w in vocab):
            sum += math.log(prob[w])
    return (math.log(frac) + sum)



In [3]:
# Multinomial naive bayes bigrams!
def fit_params_bi(num_examples, data_labels,update_vocab):
    train_no = num_examples
    # extracting the bigrams
    if(update_vocab == 1):
        vocab_no = train_no
    else:
        vocab_no = 700

    all_bi = defaultdict(int)
    for i in range(vocab_no):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        for pair in bigrams(r.split()):
            all_bi[pair]=1
                
    pos_wordCount_mult_bi = defaultdict(int)
    neg_wordCount_mult_bi = defaultdict(int)
    
    for pair in all_bi:
        pos_wordCount_mult_bi[pair] = 0
        neg_wordCount_mult_bi[pair] = 0
    numberpos_mult_bi = 0.0
    numberneg_mult_bi = 0.0
    posexamples = 0.0
    negexamples = 0.0
    # calculating the probabilites
    for i in range(train_no):
        if(data_labels[i] == 1):
            posexamples += 1
        else:
            negexamples += 1
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for pair in bigrams(bag):
            if(pair in all_bi):
                if(data_labels[i] == 1):
                    pos_wordCount_mult_bi[pair] += 1
                    numberpos_mult_bi += 1
                else:
                    neg_wordCount_mult_bi[pair] += 1
                    numberneg_mult_bi += 1
                    
    posprob = posexamples/(posexamples + negexamples)
    negprob = negexamples/(posexamples + negexamples)
    
    pos_wordprob_mult_bi = defaultdict(float)
    neg_wordprob_mult_bi = defaultdict(float)

    for pair in all_bi:
        pos_wordprob_mult_bi[pair] = 1.0*(pos_wordCount_mult_bi[pair] + 1)/(len(all_bi) + numberpos_mult_bi)
        neg_wordprob_mult_bi[pair] = 1.0*(neg_wordCount_mult_bi[pair] + 1)/(len(all_bi) + numberneg_mult_bi)
    
    return all_bi, pos_wordprob_mult_bi,neg_wordprob_mult_bi,posprob, negprob


import math
def predpos_mult_bi(words,vocab,prob, frac):
    sum = 0.0
    for pair in bigrams(words):
        if(pair in vocab):
            sum += math.log(prob[pair])
    return (sum)
            
def predneg_mult_bi(words, vocab, prob, frac):
    sum = 0.0
    for pair in bigrams(words):
        if(pair in vocab):
            sum += math.log(prob[pair])
    return (sum)



In [4]:

def train_NB(num_examples,data_labels,update_vocab):
    vocab, pp, nn, p, n = fit_params(num_examples,data_labels,update_vocab)
    all_bi, pp_bi, nn_bi, p, n = fit_params_bi(num_examples,data_labels,update_vocab)
    predictions_mult = []
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words,vocab,pp,p) #+ predpos_mult_bi(words,all_bi,pp_bi,p)
        y = predneg_mult(words,vocab,nn,n) #+ predneg_mult_bi(words,all_bi,nn_bi,n)
        if(x > y):
            predictions_mult.append(1)
        else:
            predictions_mult.append(0)
    return predictions_mult


In [5]:
# functions for accuracy prediction...
def print_accuracies(predictions, no_train,verbatim):
    train_accuracy = 0.0
    test_accuracy = 0.0
    rate = 0.0
    neg_examples = 0.0
    for i in range(700):
        if(i>=no_train and labels[i] == 0):
            neg_examples += 1
        if(predictions[i] == labels[i]):
            if(labels[i] == 0 and i>=no_train):
                rate += 1
            if i<no_train:
                train_accuracy += 1
            else:
                test_accuracy += 1
                
        elif(i>=no_train  and i<700 and verbatim == 1):
            print comments[i], "True label:",labels[i],"Predicted label:",predictions[i]
    print "\n","Test accuracy:",test_accuracy/(700-no_train),"Train accuracy:", train_accuracy/no_train, "frac negatives identified:",(rate + 0.0001)/(neg_examples+0.0001)

In [19]:
predictions = train_NB(600,labels,1)
print_accuracies(predictions,600,0)


Test accuracy: 0.86 Train accuracy: 0.98 frac negatives identified: 0.333336507921


In [13]:
# Semi supervised learning, Some kind of expectation maximization...
# Refer to https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Semi-supervised_parameter_estimation
# running the iterative process a fixed number of times.

num_iter = 1

while(num_iter < 10):
    print "iteration number:",num_iter
    predictions = train_NB(len(comments),predictions,0)
    print_accuracies(predictions,600,0)
    num_iter += 1


iteration number: 1

Test accuracy: 0.943333333333 Train accuracy: 0.976666666667 frac negatives identified: 0.729167230902
iteration number: 2

Test accuracy: 0.966666666667 Train accuracy: 0.981666666667 frac negatives identified: 0.895833550347
iteration number: 3

Test accuracy: 0.963333333333 Train accuracy: 0.973333333333 frac negatives identified: 0.916666840277
iteration number: 4

Test accuracy: 0.963333333333 Train accuracy: 0.973333333333 frac negatives identified: 0.916666840277
iteration number: 5

Test accuracy: 0.96 Train accuracy: 0.973333333333 frac negatives identified: 0.916666840277
iteration number: 6

Test accuracy: 0.96 Train accuracy: 0.971666666667 frac negatives identified: 0.916666840277
iteration number: 7

Test accuracy: 0.96 Train accuracy: 0.971666666667 frac negatives identified: 0.916666840277
iteration number: 8

Test accuracy: 0.956666666667 Train accuracy: 0.971666666667 frac negatives identified: 0.916666840277
iteration number: 9

Test accuracy: 0.

In [124]:
print_accuracies(predictions,600,1)

I see absolutely no reason for expanding the opportunities F-1 nonimmigrant students who merely make it much harder for United States young people who have such a difficult time finding work. Please do not allow additional visas or additional time for F-1 individuals to work in the U.S. True label: 0 Predicted label: 1
While some people arguing that extending OPT will put more Americans out of work, the simple truth is that you cannot ask a farmer to do the job of a painter, and you cannot ask a painter to do the job of an engineer. To be honest, highly educated people with good personality will never have trouble finding a job, regardless of being an American or a foreign student. Those who has trouble finding a job is people who have limited skills or bad personalities. The way to put more Americans at work is to EDUCATE them, not by limiting those who are ALREADY educated. If I am the company manager, if I cannot find the workers with the required skills because you SENT them home, 

In [40]:
# doing time analysis

pos_date_count = defaultdict(int)
neg_date_count = defaultdict(int)
    
def predictAllComments(num_examples,data_labels,update_vocab):
    
    vocab, pp, nn, p, n = fit_params(num_examples,data_labels,update_vocab)
    all_bi, pp_bi, nn_bi, p, n = fit_params_bi(num_examples,data_labels,update_vocab)
    predictions_mult = []
    
    print 'starting'
    count = 0
    
    for i in read('opt_dataset/opt.json'):
        #count += 1
        #if (count % 1000 == 0): print count
        comment = i['comment']
        date = i['recievedDate']
        r = ''.join([c for c in comment.lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words,vocab,pp,p) #+ predpos_mult_bi(words,all_bi,pp_bi,p)
        y = predneg_mult(words,vocab,nn,n) #+ predneg_mult_bi(words,all_bi,nn_bi,n)
        if(x > y):
            pos_date_count[date] += 1
        else:
            neg_date_count[date] += 1

In [42]:
predictAllComments(600,labels,1)
    

starting


SyntaxError: invalid syntax (<string>, line 1)

In [8]:
# Graph plot for time analysis
from matplotlib.dates import drange, num2date
import matplotlib.pyplot as plt
from datetime import  datetime
from datetime import timedelta
import numpy as np
import matplotlib.dates as mdates

pos_dict = defaultdict(int)
neg_dict = defaultdict(int)

for key in pos_date_count:
    d = datetime.strptime(key, '%m/%d/%Y').date()
    pos_dict[d] = pos_date_count[key]
    
for key in neg_date_count:
    d = datetime.strptime(key, '%m/%d/%Y').date()
    neg_dict[d] = neg_date_count[key]

In [9]:
list_of_dates = list(pos_dict.keys())

print(min(list_of_dates))
print(max(list_of_dates))

2015-10-16
2015-11-23


In [14]:
dates = num2date(drange(min(list_of_dates), max(list_of_dates) + timedelta(1), timedelta(1)))

pos_counts = np.asarray([pos_dict.get(d.date(), 0) for d in dates])
neg_counts = np.asarray([neg_dict.get(d.date(), 0) for d in dates])

frac = []
for i in range(len(pos_counts)):
    tot = pos_counts[i] + neg_counts[i]
    if(tot == 0):
        frac.append(0)
    else:
        val = pos_counts[i] * 1.0 /tot
        frac.append(val)
#print dates
#print pos_counts
#print neg_counts
#step(dates, pos_counts)
#step(dates, neg_counts)

In [20]:
print frac

[1.0, 0, 0, 1.0, 0.99809523809523815, 0.99195710455764075, 0.99715909090909094, 1.0, 1.0, 0.9876160990712074, 0.97041420118343191, 0.96842105263157896, 0.97468354430379744, 0.96296296296296291, 0.62487852283770651, 0, 0, 0.63244047619047616, 0.78174603174603174, 0.93805309734513276, 0.8177905308464849, 0.60151324085750313, 0, 0, 0.65180102915951976, 0.73307729236677865, 0.95958624007697857, 0.97975010771219306, 0.98942682248191427, 0.98011363636363635, 0.97247706422018354, 0.98970133882595268, 0.95699075371252451, 0.83333333333333337, 0.94402985074626866, 0.96463993100474343, 0.9668587896253602, 0, 0.66666666666666663]


In [23]:
# Plot
plt.close('all')
fig, ax = plt.subplots()
plt.xlabel('Posted Date', fontsize=12)
plt.ylabel('# Positive Fraction', fontsize=12)
date_format = mdates.DateFormatter("%m/%d/%Y")
ax.xaxis.set_major_formatter(date_format)
ax.autoscale_view()
width = 0.35
plpos = ax.bar(dates, pos_counts, width, color='b', label="positive")
plneg = ax.bar(dates, neg_counts, width, color='r', label="negative")
#plt.legend()
plt.show()
plt.close('all')